In [1]:
import xarray as xr
import obsarray

In [2]:
# Read Dataset contents

ds = xr.open_dataset("obs_example.nc")

In [3]:
# Inspect Dataset contents

print("Observation", ds.unc.obs_vars)   # Variables with uncertainties
print("Uncertainty", ds.unc.unc_vars)   # Uncertainty variables

Observation Data variables:
    temperature  (x, y, time) float64 ...
    pressure     (x, y, time) float64 ...
Uncertainty Data variables:
    u_ran_temperature  (x, y, time) float64 ...
    u_str_temperature  (x, y, time) float64 ...
    u_ran_pressure     (x, y, time) float64 ...


In [4]:
# Inspect uncertainty variables for a particular variable

print(ds.unc["temperature"])

<VariableUncertainty>
Variable Uncertainties: 'temperature'
Data variables:
    u_ran_temperature  (x, y, time) float64 ...
    u_str_temperature  (x, y, time) float64 ...


In [5]:
# Add uncertainty variable 

# Define uncertainty variable as tuple: (dims, data, attrs) 
# where attrs should include:
# > "err_corr" - error-correlation structure defintion (if omitted assumed random)
# > "pdf_shape" - uncertainty pdf shape (if omitted assumed gaussian)

ds.unc["temperature"]["u_sys_temperature"] = (
    ["x", "y", "time"],
    ds.temperature * 0.03,
	{
        "err_corr": [
            {
                "dim": "x",
                "form": "systematic",
                "params": [],
            },
            {
                "dim": "y",
                "form": "systematic",
                "params": [],
            },
            {
                "dim": "time",
                "form": "systematic",
                "params": [],
            }
        ]
    },
)

# Now inspect again

print(ds.unc["temperature"])



<VariableUncertainty>
Variable Uncertainties: 'temperature'
Data variables:
    u_ran_temperature  (x, y, time) float64 ...
    u_str_temperature  (x, y, time) float64 ...
    u_sys_temperature  (x, y, time) float64 0.5091 0.1441 ... 0.5432 0.3479


In [6]:
# Get total variable uncertainty

ds.unc["temperature"].total

<xarray.DataArray (x: 2, y: 2, time: 3)>
array([[[1.11277669, 0.31500626, 1.78822662],
        [1.51663088, 1.22266768, 1.37061312]],

       [[1.24821081, 0.58453532, 0.3093862 ],
        [0.39865853, 1.18741975, 0.76051039]]])
Dimensions without coordinates: x, y, time

In [7]:
# Get total variable random uncertainty

ds.unc["temperature"].random

<xarray.DataArray 'u_ran_temperature' (x: 2, y: 2, time: 3)>
array([[[0.84848427, 0.24019002, 1.36351001],
        [1.1564202 , 0.93227537, 1.04508271]],

       [[0.95175182, 0.445704  , 0.23590476],
        [0.30397428, 0.90539907, 0.57988374]]])
Dimensions without coordinates: x, y, time

In [8]:
# Get total variable systematic uncertainty

ds.unc["temperature"].systematic

<xarray.DataArray 'u_sys_temperature' (x: 2, y: 2, time: 3)>
array([[[0.50909056, 0.14411401, 0.81810601],
        [0.69385212, 0.55936522, 0.62704963]],

       [[0.57105109, 0.2674224 , 0.14154286],
        [0.18238457, 0.54323944, 0.34793024]]])
Dimensions without coordinates: x, y, time

In [9]:
# Inspect uncertainty variables
for unc_var in ds.unc["temperature"]:
    print(unc_var, "\n")

<Uncertainty> 
'u_ran_temperature' (x: 2, y: 2, time: 3)>
array([[[0.848484, 0.24019 , 1.36351 ],
        [1.15642 , 0.932275, 1.045083]],

       [[0.951752, 0.445704, 0.235905],
        [0.303974, 0.905399, 0.579884]]])
Dimensions without coordinates: x, y, time
Attributes: (12/13)
    err_corr_1_dim:     x
    err_corr_1_form:    random
    err_corr_1_units:   []
    err_corr_1_params:  []
    err_corr_2_dim:     y
    err_corr_2_form:    random
    ...                 ...
    err_corr_2_params:  []
    err_corr_3_dim:     time
    err_corr_3_form:    random
    err_corr_3_units:   []
    err_corr_3_params:  []
    pdf_shape:          gaussian 

<Uncertainty> 
'u_str_temperature' (x: 2, y: 2, time: 3)>
array([[[0.509091, 0.144114, 0.818106],
        [0.693852, 0.559365, 0.62705 ]],

       [[0.571051, 0.267422, 0.141543],
        [0.182385, 0.543239, 0.34793 ]]])
Dimensions without coordinates: x, y, time
Attributes: (12/13)
    err_corr_1_dim:     x
    err_corr_1_form:    custom
 

In [10]:
# Test if uncertainty fully random in all dimensions
print("u_ran_temperature random: ", ds.unc["temperature"]["u_ran_temperature"].is_random)
print("u_str_temperature random: ", ds.unc["temperature"]["u_str_temperature"].is_random)
print("u_sys_temperature random: ", ds.unc["temperature"]["u_sys_temperature"].is_random)

# Test if uncertainty fully systematic in all dimensions
print("u_ran_temperature systematic: ", ds.unc["temperature"]["u_ran_temperature"].is_systematic)
print("u_str_temperature random: ", ds.unc["temperature"]["u_str_temperature"].is_systematic)
print("u_sys_temperature systematic: ", ds.unc["temperature"]["u_sys_temperature"].is_systematic)

u_ran_temperature random:  True
u_str_temperature random:  False
u_sys_temperature random:  False
u_ran_temperature systematic:  False
u_str_temperature random:  False
u_sys_temperature systematic:  True


In [12]:
# Get uncertainty info for given variable

# data (return data array)
print("Data:\n", ds.unc["temperature"]["u_str_temperature"].data)

# pdf shape
print("\nPDF Shape:\n", ds.unc["temperature"]["u_str_temperature"].pdf_shape)

# for error-correlation, error-correlation interface object returned
print("\nError Correlation:\n", ds.unc["temperature"]["u_str_temperature"].err_corr)

Data:
 <xarray.DataArray 'u_str_temperature' (x: 2, y: 2, time: 3)>
array([[[0.509091, 0.144114, 0.818106],
        [0.693852, 0.559365, 0.62705 ]],

       [[0.571051, 0.267422, 0.141543],
        [0.182385, 0.543239, 0.34793 ]]])
Dimensions without coordinates: x, y, time
Attributes: (12/13)
    err_corr_1_dim:     x
    err_corr_1_form:    custom
    err_corr_1_units:   []
    err_corr_1_params:  err_corr_str_temperature_x
    err_corr_2_dim:     y
    err_corr_2_form:    systematic
    ...                 ...
    err_corr_2_params:  []
    err_corr_3_dim:     time
    err_corr_3_form:    systematic
    err_corr_3_units:   []
    err_corr_3_params:  []
    pdf_shape:          gaussian

PDF Shape:
 gaussian

Error Correlation:
 <ErrCorr 'u_str_temperature'> 
* 'dim': 'time', 'form': 'systematic', 'params': array([], dtype=float64), 'units': array([], dtype=float64)
* 'dim': 'x', 'form': 'custom', 'params': 'err_corr_str_temperature_x', 'units': array([], dtype=float64)
* 'dim': 'y', 

In [13]:
# Inspect error-correlation information
ds.unc["temperature"]["u_str_temperature"].err_corr.params

[{'dim': 'time',
  'form': 'systematic',
  'params': array([], dtype=float64),
  'units': array([], dtype=float64)},
 {'dim': 'x',
  'form': 'custom',
  'params': 'err_corr_str_temperature_x',
  'units': array([], dtype=float64)},
 {'dim': 'y',
  'form': 'systematic',
  'params': array([], dtype=float64),
  'units': array([], dtype=float64)}]

In [14]:
# Inspect error-correlation information for a particular dimension
ds.unc["temperature"]["u_str_temperature"].err_corr.dim_params("x")

{'dim': 'x',
 'form': 'custom',
 'params': 'err_corr_str_temperature_x',
 'units': array([], dtype=float64)}

In [15]:
# Build error-correlation matrix (ordered as ds.variable.flatten())
ds.unc["temperature"]["u_ran_temperature"].err_corr.to_matrix()

TypeError: to_matrix() missing 1 required positional argument: 'dim'

In [16]:
# Build error-correlation matrix or for one dimension
# (or set of dimensions if error-correlation defintion spans multiple dimensions)

ds.unc["temperature"]["u_str_temperature"].err_corr.to_matrix(dim="x")

KeyError: 'e'